In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install rdkit

In [2]:
import os
import sys
import torch
import time
from tqdm import tqdm
import datetime
sys.path.append("/content/drive/MyDrive/HeckLit")
from utils.rxn import *
from utils.molecule import *
from utils.dataset_analysis import *
from models.ANN import *
import matplotlib.pyplot as plt

In [ ]:
rs_list = [1,2,3,4,5]
for rs in rs_list:
  # 1. import data
  data = pd.read_excel("/content/drive/MyDrive/HeckLit/data/Heck/Heck_fp.xlsx")
  random_state = rs
  data = data.sample(random_state=random_state, frac=1).reset_index(drop=True)

  # 2. build dataset & dataloader
  rxnfp_set = list()
  drfp_set = list()
  yield_set = list()
  rxn_list = df_to_rxn_list(data)
  len_drfp = 0

  for batch in tqdm(range(data.shape[0])):
    rxn = rxn_list[batch]

    # features
    rxnfp = read_rxnfp(data.loc[batch]["rxnfp"])
    drfp = read_drfp(data.loc[batch]["drfp"])
    len_drfp = drfp.shape[0]
    # label
    y = np.array(rxn.rxn_yield / 100)

    rxnfp_set.append(rxnfp)
    drfp_set.append(drfp)
    yield_set.append(y)

  # split of train & test set
  ratio = 0.7

  # rxnfp
  batch = len(rxnfp_set)
  rxnfp_trainset = [rxnfp_set[0: int(ratio * batch)], yield_set[0: int(ratio * batch)]]
  rxnfp_testset = [rxnfp_set[int(ratio * batch) + 1:], yield_set[int(ratio * batch) + 1:]]

  # drfp
  batch = len(drfp_set)
  drfp_trainset = [drfp_set[0: int(ratio * batch)], yield_set[0: int(ratio * batch)]]
  drfp_testset = [drfp_set[int(ratio * batch) + 1:], yield_set[int(ratio * batch) + 1:]]

  # report
  dir_path = "/content/drive/MyDrive/HeckLit/exp/Heck/Heck_ML_%s" % datetime.datetime.now()
  os.mkdir("%s" % dir_path)
  f = open("%s/Model_Training_Report.txt" % dir_path, mode="w")

  # record
  f.write("params:\n")
  f.write("random_state=%s\n" % random_state)
  f.write("ratio=%s\n" % ratio)
  f.write("\n")

  # 3.Machine Learning methods
  # RandomForest
  from sklearn.ensemble import RandomForestRegressor

  # Train
  print("RandomForest Start Training")
  start = time.time()

  rxnfp_rf = RandomForestRegressor(n_estimators=150, random_state=0)
  drfp_rf = RandomForestRegressor(n_estimators=150, random_state=0)
  rxnfp_rf.fit(rxnfp_trainset[0], rxnfp_trainset[1])
  drfp_rf.fit(drfp_trainset[0], drfp_trainset[1])

  print("Finish Training")
  print("Training Time: %.2f s" % (time.time() - start))  # 截止时间

  # Eval
  # trainset
  # R2
  rxnfp_train_R2 = R2(rxnfp_rf.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_R2 = R2(drfp_rf.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))
  # RMSE
  rxnfp_train_RMSE = RMSE(rxnfp_rf.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_RMSE = RMSE(drfp_rf.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))

  # R2
  rxnfp_test_R2 = R2(rxnfp_rf.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_R2 = R2(drfp_rf.predict(drfp_testset[0]), np.array(drfp_testset[1]))
  # RMSE
  rxnfp_test_RMSE = RMSE(rxnfp_rf.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_RMSE = RMSE(drfp_rf.predict(drfp_testset[0]), np.array(drfp_testset[1]))

  # Record
  f.write("RandomForest:\n")
  f.write("params:\n")
  f.write("rxnfp RF:%s\n" % rxnfp_rf.n_estimators)
  f.write("drfp RF:%s\n" % drfp_rf.n_estimators)

  f.write("rxnfp:\n")
  f.write("rxnfp_train_R2=%s\n" % rxnfp_train_R2)
  f.write("rxnfp_train_RMSE=%s\n" % rxnfp_train_RMSE)
  f.write("rxnfp_test_R2=%s\n" % rxnfp_test_R2)
  f.write("rxnfp_test_RMSE=%s\n" % rxnfp_test_RMSE)
  f.write("drfp:\n")
  f.write("drfp_train_R2=%s\n" % drfp_train_R2)
  f.write("drfp_train_RMSE=%s\n" % drfp_train_RMSE)
  f.write("drfp_test_R2=%s\n" % drfp_test_R2)
  f.write("drfp_test_RMSE=%s\n" % drfp_test_RMSE)
  f.write("\n")

  # RF Figure
  # RXNFP
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(rxnfp_testset[1]).flatten() * 100
  pr = rxnfp_rf.predict(rxnfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("RF RXNFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/RF RXNFP Performance Figure.png" % dir_path)
  plt.show()

  # DRFP
  # Figure
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(drfp_testset[1]).flatten() * 100
  pr = drfp_rf.predict(drfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("RF DRFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/RF DRFP Performance Figure.png" % dir_path)
  plt.show()

  # xgboost
  from xgboost import XGBRegressor

  # Train
  print("XGBoost Start Training")
  start = time.time()

  rxnfp_xgb = XGBRegressor(n_estimators=300)
  drfp_xgb = XGBRegressor(n_estimators=300)
  rxnfp_xgb.fit(rxnfp_trainset[0], rxnfp_trainset[1])
  drfp_xgb.fit(drfp_trainset[0], drfp_trainset[1])

  print("Finish Training")
  print("Training Time: %.2f s" % (time.time() - start))  # 截止时间

  # Eval
  # trainset
  # R2
  rxnfp_train_R2 = R2(rxnfp_xgb.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_R2 = R2(drfp_xgb.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))
  # RMSE
  rxnfp_train_RMSE = RMSE(rxnfp_xgb.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_RMSE = RMSE(drfp_xgb.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))

  # R2
  rxnfp_test_R2 = R2(rxnfp_xgb.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_R2 = R2(drfp_xgb.predict(drfp_testset[0]), np.array(drfp_testset[1]))
  # RMSE
  rxnfp_test_RMSE = RMSE(rxnfp_xgb.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_RMSE = RMSE(drfp_xgb.predict(drfp_testset[0]), np.array(drfp_testset[1]))

  # Record
  f.write("XGBoost:\n")
  f.write("params:\n")
  f.write("rxnfp XGB:%s\n" % rxnfp_xgb.n_estimators)
  f.write("drfp XGB:%s\n" % drfp_xgb.n_estimators)

  f.write("rxnfp:\n")
  f.write("rxnfp_train_R2=%s\n" % rxnfp_train_R2)
  f.write("rxnfp_train_RMSE=%s\n" % rxnfp_train_RMSE)
  f.write("rxnfp_test_R2=%s\n" % rxnfp_test_R2)
  f.write("rxnfp_test_RMSE=%s\n" % rxnfp_test_RMSE)
  f.write("drfp:\n")
  f.write("drfp_train_R2=%s\n" % drfp_train_R2)
  f.write("drfp_train_RMSE=%s\n" % drfp_train_RMSE)
  f.write("drfp_test_R2=%s\n" % drfp_test_R2)
  f.write("drfp_test_RMSE=%s\n" % drfp_test_RMSE)
  f.write("\n")

  # XGB Figure
  # RXNFP
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(rxnfp_testset[1]).flatten() * 100
  pr = rxnfp_xgb.predict(rxnfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("XGB RXNFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/XGB RXNFP Performance Figure.png" % dir_path)
  plt.show()

  # DRFP
  # Figure
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(drfp_testset[1]).flatten() * 100
  pr = drfp_xgb.predict(drfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("XGB DRFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/XGB DRFP Performance Figure.png" % dir_path)
  plt.show()

  # Supporting Vector Machine
  from sklearn import svm
  from sklearn.svm import SVR

  rxnfp_svm = SVR(kernel='rbf', C=1.25)
  drfp_svm = SVR(kernel='rbf', C=1.25)
  rxnfp_svm.fit(rxnfp_trainset[0], rxnfp_trainset[1])
  drfp_svm.fit(drfp_trainset[0], drfp_trainset[1])

  print("Finish Training")
  print("Training Time: %.2f s" % (time.time() - start))  # 截止时间

  # Eval
  # trainset
  # R2
  rxnfp_train_R2 = R2(rxnfp_svm.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_R2 = R2(drfp_svm.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))
  # RMSE
  rxnfp_train_RMSE = RMSE(rxnfp_svm.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_RMSE = RMSE(drfp_svm.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))

  # R2
  rxnfp_test_R2 = R2(rxnfp_svm.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_R2 = R2(drfp_svm.predict(drfp_testset[0]), np.array(drfp_testset[1]))
  # RMSE
  rxnfp_test_RMSE = RMSE(rxnfp_svm.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_RMSE = RMSE(drfp_svm.predict(drfp_testset[0]), np.array(drfp_testset[1]))

  # Record
  f.write("Supporting Vector Machine:\n")

  f.write("rxnfp:\n")
  f.write("rxnfp_train_R2=%s\n" % rxnfp_train_R2)
  f.write("rxnfp_train_RMSE=%s\n" % rxnfp_train_RMSE)
  f.write("rxnfp_test_R2=%s\n" % rxnfp_test_R2)
  f.write("rxnfp_test_RMSE=%s\n" % rxnfp_test_RMSE)
  f.write("drfp:\n")
  f.write("drfp_train_R2=%s\n" % drfp_train_R2)
  f.write("drfp_train_RMSE=%s\n" % drfp_train_RMSE)
  f.write("drfp_test_R2=%s\n" % drfp_test_R2)
  f.write("drfp_test_RMSE=%s\n" % drfp_test_RMSE)
  f.write("\n")

  # SVM Figure
  # RXNFP
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(rxnfp_testset[1]).flatten() * 100
  pr = rxnfp_svm.predict(rxnfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("SVM RXNFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/SVM RXNFP Performance Figure.png" % dir_path)
  plt.show()

  # DRFP
  # Figure
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(drfp_testset[1]).flatten() * 100
  pr = drfp_svm.predict(drfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("SVM DRFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/SVM DRFP Performance Figure.png" % dir_path)
  plt.show()

  # kNN
  from sklearn.neighbors import KNeighborsRegressor

  # Train
  print("KNeighborsRegressor Start Training")
  start = time.time()

  rxnfp_knn = KNeighborsRegressor(n_neighbors=50)
  drfp_knn = KNeighborsRegressor(n_neighbors=50)
  rxnfp_knn.fit(rxnfp_trainset[0], rxnfp_trainset[1])
  drfp_knn.fit(drfp_trainset[0], drfp_trainset[1])

  print("Finish Training")
  print("Training Time: %.2f s" % (time.time() - start))  # 截止时间

  # Eval
  # trainset
  # R2
  rxnfp_train_R2 = R2(rxnfp_knn.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_R2 = R2(drfp_knn.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))
  # RMSE
  rxnfp_train_RMSE = RMSE(rxnfp_knn.predict(rxnfp_trainset[0]), np.array(rxnfp_trainset[1]))
  drfp_train_RMSE = RMSE(drfp_knn.predict(drfp_trainset[0]), np.array(drfp_trainset[1]))

  # R2
  rxnfp_test_R2 = R2(rxnfp_knn.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_R2 = R2(drfp_knn.predict(drfp_testset[0]), np.array(drfp_testset[1]))
  # RMSE
  rxnfp_test_RMSE = RMSE(rxnfp_knn.predict(rxnfp_testset[0]), np.array(rxnfp_testset[1]))
  drfp_test_RMSE = RMSE(drfp_knn.predict(drfp_testset[0]), np.array(drfp_testset[1]))

  # Record
  f.write("KNeighbors Regressor:\n")
  f.write("params:\n")
  f.write("rxnfp KNN:%s\n" % rxnfp_knn.n_neighbors)
  f.write("drfp KNN:%s\n" % drfp_knn.n_neighbors)

  f.write("rxnfp:\n")
  f.write("rxnfp_train_R2=%s\n" % rxnfp_train_R2)
  f.write("rxnfp_train_RMSE=%s\n" % rxnfp_train_RMSE)
  f.write("rxnfp_test_R2=%s\n" % rxnfp_test_R2)
  f.write("rxnfp_test_RMSE=%s\n" % rxnfp_test_RMSE)
  f.write("drfp:\n")
  f.write("drfp_train_R2=%s\n" % drfp_train_R2)
  f.write("drfp_train_RMSE=%s\n" % drfp_train_RMSE)
  f.write("drfp_test_R2=%s\n" % drfp_test_R2)
  f.write("drfp_test_RMSE=%s\n" % drfp_test_RMSE)
  f.write("\n")

  # KNN Figure
  # RXNFP
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(rxnfp_testset[1]).flatten() * 100
  pr = rxnfp_knn.predict(rxnfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("KNN RXNFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/KNN RXNFP Performance Figure.png" % dir_path)
  plt.show()

  # DRFP
  # Figure
  fig = plt.figure(dpi=120, figsize=(20, 7))
  # Test set performance
  plt.subplot(1, 2, 1)
  tr = np.array(drfp_testset[1]).flatten() * 100
  pr = drfp_knn.predict(drfp_testset[0]).flatten() * 100
  plt.scatter(pr, tr, alpha=0.7, marker=".")
  plt.xlabel("Predicted Yield", fontsize=10)
  plt.ylabel("Observed Yield", fontsize=10)
  x = np.linspace(0, 100, 100)
  y = np.linspace(0, 100, 100)
  plt.plot(x, y, linestyle="--", color="r")
  plt.title("Test set performance", fontsize=15)

  # Error Distrubution
  plt.subplot(1, 2, 2)
  x_e = tr
  l = x_e.shape[0]
  y_e = list()
  for i in range(x_e.shape[0]):
    y_e.append(RMSE(pr[i], tr[i]))
  plt.bar(x_e, y_e)
  plt.xlabel("Observed Yield", fontsize=10)
  plt.ylabel("RMSE Error", fontsize=10)
  plt.xticks([0, 50, 100])
  plt.ylim(0, max(y_e) + 10)
  plt.title("Error Distrubution", fontsize=15)

  fig.suptitle("KNN DRFP for Heck dataset", fontsize=16)
  plt.tight_layout()
  plt.savefig("%s/KNN DRFP Performance Figure.png" % dir_path)
  plt.show()

  f.close()
